In [1]:
!pip install finance-datareader

In [2]:
!pip install opendartreader

In [3]:
!pip install pykrx

     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 256kB 29.5MB/s 


In [4]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import OpenDartReader
import datetime
from pykrx import stock

In [5]:
api_key = '5729a47b92ddd434526831ee99ff01f65e6434db'
dart = OpenDartReader(api_key) 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
file = pd.read_excel('/content/drive/MyDrive/소회의실 프로젝트/2021 금융데이터 경진대회_샘플_한국투자증권.xlsx')

In [ ]:
# fdr.StockListing('KRX')

In [8]:
print(file.columns)

Index(['고객번호', '계좌번호', '고객성별구분코드', '동일나이군구분코드', '주소(시도)', '주소(시군구)', '고객등록일자',
       '계좌개설일자', '주문일자', '주문시각', '최초주문번호', '주문번호', '상품번호', '상품명', '거래소구분코드',
       '매도매수구분코드', '주문구분코드', '한국거래소호가조건코드', '실주문단가', '주문수량', '정정취소구분코드',
       '정정확인수량', '취소확인수량', '총체결건수', '총체결수량', '총체결금액', '반대매매여부', '예약주문여부',
       '대출유형코드', '대출일자', '주문증거금율', '전일종가', '당일시가', '일중고가', '일중저가', '당일종가',
       '거래수량', '거래대금', '상한가여부', '하한가여부'],
      dtype='object')


In [9]:
## 주문일시
file['주문일자'] = file['주문일자'].astype('str')
file['주문시각']= file['주문시각'].astype(int).apply(lambda x: '{0:06d}'.format(x))
date_time = file['주문일자']+file['주문시각'].astype('str')
file['주문일시']= pd.to_datetime(date_time)

In [10]:
##주소
file['주소'] = file['주소(시도)']+' '+file['주소(시군구)']

In [11]:
customer_info = pd.DataFrame(file.groupby(['고객번호','고객성별구분코드','동일나이군구분코드','주소'])['상품명'].nunique()).reset_index()
customer_info= customer_info.set_index('고객번호')

### 1. 거래수량 :총 매수수량
### 2. 거래액: 총 매수액


In [12]:
#구매자별 매수금액
customer_purchase = file.loc[file['매도매수구분코드']==2,:].groupby(['고객번호'])['총체결금액'].sum() 
customer_amount = file.loc[file['매도매수구분코드']==2,:].groupby(['고객번호'])['총체결수량'].sum()

# 코드 수정함
customer_info['총매수금액']= customer_purchase
customer_info['총매수수량'] = customer_amount


### 3. 투자상품의 종류별 비중(총매수금액을 가중치로) 
ETF, 대형주, 중형주, 소형주, KOSDAQ 


In [13]:
kosdaq_tickers = stock.get_market_ticker_list("20200120", market="KOSDAQ")
kospi_tickers = stock.get_market_ticker_list("20200120", market="KOSPI")
kospi_caps  = stock.get_market_cap_by_ticker("20200120").loc[kospi_tickers,'시가총액'].sort_values(ascending= False)
kosdaq_caps  = stock.get_market_cap_by_ticker("20200120").loc[kosdaq_tickers,'시가총액'].sort_values(ascending= False).index
etf_cap = stock.get_etf_ticker_list("20200120")

large_cap = kospi_caps[:100].index 
medium_cap = kospi_caps[100:300].index
small_cap = kospi_caps[300:].index

In [14]:
#stock_type
file['stock_type']= ' '
# file.loc[file['상품명'].str.contains('투자신탁')== True, 'stock_type'] = 'ETF'
file.loc[file['상품번호'].apply(lambda x: x[6:]).isin(etf_cap), 'stock_type'] ='ETF'
file.loc[file['상품번호'].apply(lambda x: x[6:]).isin(large_cap), 'stock_type'] ='large_cap'
file.loc[file['상품번호'].apply(lambda x: x[6:]).isin(medium_cap), 'stock_type'] ='medium_cap'
file.loc[file['상품번호'].apply(lambda x: x[6:]).isin(small_cap), 'stock_type'] ='small_cap'
file.loc[file['상품번호'].apply(lambda x: x[6:]).isin(kosdaq_caps), 'stock_type'] ='kosdaq_cap'

In [15]:
#종목타입별 매수금액 비중

#구매자별 상품별 매수금액
product_purchase = file.loc[file['매도매수구분코드']==2,:].groupby(['고객번호','상품번호','stock_type'])['총체결금액'].sum().unstack().fillna(0)
#구매자별 종목타입별 매수금액 비중
type_purchase = product_purchase.groupby('고객번호').sum().apply(lambda x: x/customer_purchase )
customer_info = customer_info.join(type_purchase, on='고객번호')

In [16]:
customer_info.rename(columns={'상품명':'상품개수'}, inplace=True)
# customer_info

### 4. 투자한 섹터 개수


In [17]:
# 일단 코드번호 컬럼 생성
file['code'] = file['상품번호'].str[-6:]

# 코스피 코스닥 Sector, Industry 뽑기
stocks_kospi = fdr.StockListing('KOSPI')
stocks_kosdaq = fdr.StockListing('KOSDAQ')

# 섹터 컬럼 원 데이터에 붙이기
stocks = pd.concat([stocks_kospi,stocks_kosdaq], ignore_index=True)
sector_df = stocks.loc[:,['Symbol', 'Sector', 'Industry']]
sector_df.rename(columns={'Symbol':'code'}, inplace=True)
file = pd.merge(file, sector_df, on='code')

In [18]:
# 투자한 섹터 개수
industry_df = pd.DataFrame(file.groupby(['고객번호', 'Sector', 'Industry']).size()).reset_index()
# industry_df2 = pd.DataFrame(industry_df.groupby(['고객번호'])['Industry'].size())
industry_df = pd.DataFrame(industry_df.groupby(['고객번호'])['Industry'].size()).reset_index()
industry_df2 = pd.DataFrame(file['고객번호'].unique(), columns=['고객번호'])
industry_df2 = pd.merge(industry_df, industry_df2, how='outer', on='고객번호')

In [19]:
industry_df2

,고객번호,Industry
0,C00010,9.0
1,C000100,5.0
2,C0001,NaN


In [20]:
# industry_df2.loc['C0001'] = 0 # groupby 후 C0001의 size개수가 0개 일때는 아예 행에 나오질 않아서... 하드코딩으로 억지로 만들었는데 이거 자연스럽게 어떻게 할지... ㅠㅠ
industry_df2.fillna(0, inplace=True)
industry_df2.sort_index(inplace=True) 

In [21]:
# customer_info = customer_info.join(industry_df2)
customer_info = pd.merge(customer_info, industry_df2, on='고객번호')
customer_info

,고객번호,고객성별구분코드,동일나이군구분코드,주소,상품개수,총매수금액,총매수수량,ETF,kosdaq_cap,large_cap,medium_cap,small_cap,Industry
0,C0001,1,10,경기 성남시,3,8814255,1327,1.000000,0.000000,0.000000,0.000000,0.00000,0.0
1,C00010,1,30,충남 아산시,9,51683575,23704,0.000000,0.521941,0.327249,0.000000,0.15081,9.0
2,C000100,1,50,서울 강서구,6,48478705,2487,0.202753,0.000000,0.698740,0.098507,0.00000,5.0


### 5. 종목보유기간: 매수-매도 기간 
거래량을 차감해서 계산하는 형태


In [22]:
file[['고객등록일자', '계좌개설일자', '주문일자']] = file[['고객등록일자', '계좌개설일자', '주문일자']].apply(pd.to_datetime, format='%Y%m%d')

In [23]:
# return 2가지
# 1. 매도된 주식이 언제 매수한 주식인지 계산
# 2. 매도가 안되고 남아 있는 주식의 인덱스 
# **** 거래일시가 sorting되어 있어야 작동 **** 
def first_sell_date_idx(df):
    a, b = np.array([]), np.array([])
    for idx, row in df.iterrows():
        if row['매도매수구분코드'] == 2:
            a = np.append(a, np.array(row['총체결수량']))
            b = np.append(b, np.array(0))
        elif row['매도매수구분코드'] == 1:
            b = np.append(b, np.array(row['총체결수량']))
            a = np.append(a, np.array(0))

    idx = np.array([])
    for i in range(len(a)):
        flag = True
        if b[i] == 0:
            idx = np.append(idx, np.array([-1]))
        else:
            for j in range(len(a)):
                if(b[i] <= a[j]):                  
                    a[j] = a[j] - b[i]
                    b[i] = 0
                    if flag:
                        idx = np.append(idx, np.array([j]))
                        flag = False
                    break
                elif (b[i] > a[j]) & (a[j] !=0):   
                    b[i] = b[i] - a[j]
                    a[j] = 0
                    if flag:
                        idx = np.append(idx, np.array([j]))
                        flag = False
                   
    rest_index = np.where(a != 0)[0].tolist()
    return np.array(idx, dtype = np.int64).tolist(), rest_index

In [24]:
# 각각의 주문 보유기간 계산 하는 함수
# (고객번호, 계좌번호, 상품번호)별로 묶어서 계산
def buysell(df, userID, acc_num):
    invest = df[(df['고객번호'] == userID) & (df['계좌번호'] == acc_num)][['고객번호', '계좌번호', '주문일자', '주문시각', '주문일시', '최초주문번호', '주문번호', '상품번호', '상품명', '거래소구분코드', '매도매수구분코드', '실주문단가', '주문수량', '정정취소구분코드', '정정확인수량', '취소확인수량', '총체결건수', '총체결수량', '총체결금액']]
    combine_df = pd.DataFrame(columns= invest.columns)
    
    # 매도되지 않은 주식의 보유기간 계산을 위한 기준일 
    base_date = datetime.datetime(2020, 4, 1, 15, 0, 0)
    end_march = datetime.datetime.strptime('2020-03-15', '%Y-%m-%d')
    
    # (고객번호, 계좌번호)가 거래한 상품번호
    stock_code = invest['상품번호'].unique().tolist()
    
    # loop 한번 돌 때 마다 상품번호 끼리 묶음 
    for num in stock_code:
        temp = invest.loc[invest['상품번호'] == num]
        
        # 주문취소 row 2개 삭제
        cancel_set = set(temp[temp['정정취소구분코드'] == 2]['최초주문번호'])
        temp = temp[~temp['최초주문번호'].isin(cancel_set)]
        
        # 가격 정정 --> 한줄로 (마지막으로 체결된 row만 남김) 
        correct_set = set(temp[temp['정정취소구분코드'] == 1]['최초주문번호'])
        temp = temp.drop(temp[(temp['최초주문번호'].isin(correct_set)) & (temp['총체결수량'] == 0)].index)
          
        """
        종목보유기간 계산
        - 매도시, 종목 보유 기간이 찍힘
        - 일반적으로 매도매수코드가 2일때(매수) NaN값을 가짐
        - 매도매수코드가 2인데 종목보유기간이 있는 행 --> 실제로 팔리지 않아 기준일(4월1일)에 팔았다고 가정하고 보유기간 계산
        - 3월 말 이후에 매수하고 매도되지않은 주식의 종목보유기간 --> -1 
        """
        # 총 매도수량
        sell_quantity = temp[temp['매도매수구분코드']==1]['총체결수량'].sum()
        # 총 매수수량
        buy_quantity = temp[temp['매도매수구분코드']==2]['총체결수량'].sum()
        
        # case 3가지 
        if(sell_quantity == 0): # 1. 매수만 하고 매도가 없을 때 
            temp['종목보유기간'] =  np.where(temp['주문일자'] > end_march, -1,
                                       ((base_date - temp['주문일시']).dt.days * 1440) + ((base_date - temp['주문일시']).dt.seconds/60))
        elif(sell_quantity == buy_quantity): # 2. 매수한 수량을 모두 매도 함
            temp['매수시점idx'], rest_index =  first_sell_date_idx(temp)
            temp['종목보유기간'] = np.where(temp['매수시점idx'] == -1, np.nan, 
                                       ((temp['주문일시'] - temp.iloc[temp['매수시점idx']]['주문일시'].values).dt.days * 1440) + (temp['주문일시'] - temp.iloc[temp['매수시점idx']]['주문일시'].values).dt.seconds/60)
            temp.drop('매수시점idx', axis=1, inplace=True)
        elif(buy_quantity > sell_quantity): # 3. 매수한 수량이 매도한 수량보다 많음
            temp['매수시점idx'], rest_index =  first_sell_date_idx(temp)
            temp['종목보유기간'] = np.where(temp['매수시점idx'] == -1, np.nan, 
                                       ((temp['주문일시'] - temp.iloc[temp['매수시점idx']]['주문일시'].values).dt.days * 1440) + (temp['주문일시'] - temp.iloc[temp['매수시점idx']]['주문일시'].values).dt.seconds/60)
            # 매도되지 못한 나머지
            for idx in rest_index:
                temp.iloc[idx, temp.columns.get_loc('종목보유기간')] = np.where(temp.iloc[idx, temp.columns.get_loc('주문일자')] > end_march, np.nan, 
                                                                          ((base_date - temp.iloc[idx, temp.columns.get_loc('주문일시')]).days * 1440) + (base_date - temp.iloc[idx, temp.columns.get_loc('주문일시')]).seconds/60)
            temp.drop('매수시점idx', axis=1, inplace=True)
        combine_df = pd.concat([combine_df, temp])
        
    combine_df = combine_df.sort_index()    
    return combine_df        


In [25]:
def for_grouping(x):   
    if (~x.isin([-1])).sum() == 0: # 모든 종목보유기간이 -1 
        avg = -1
    else:
        avg = x[x >= 0].mean()
   
    return avg
    

# 고객의 평균 보유기간 계산 하는 함수    
def cal_term(df): 
    cust_acc_set = set(zip(df['고객번호'], df['계좌번호']))
    cust_cap_term = pd.DataFrame(columns=['고객번호', 'etf보유기간', '우량주보유기간', '중형주보유기간', '소형주보유기간', 'kosdaq보유기간'])
    for cust, acc in cust_acc_set:
        # 매매 개별 종목보유기간 불러오기
        temp = buysell(df, cust, acc)[['고객번호', '상품번호', '종목보유기간']]

        # ** 수정하기 **
        # 회사규모로 나눔, 나중에 코드 합칠 때, 컬럼으로 만들어 진걸로 수정
        temp['규모'] = ''
        temp['상품번호'] = temp['상품번호'].apply(lambda x: x[6:])
        temp.loc[temp['상품번호'].isin(large_cap), '규모'] = '우량주보유기간'
        temp.loc[temp['상품번호'].isin(medium_cap), '규모'] = '중형주보유기간'
        temp.loc[temp['상품번호'].isin(small_cap), '규모'] = '소형주보유기간'
        temp.loc[temp['상품번호'].isin(kosdaq_caps), '규모'] = 'kosdaq보유기간'
        temp.loc[temp['상품번호'].isin(etf_cap), '규모'] = 'etf보유기간'

        # 그룹핑 하고 평균 내기, 모든 종목 보유기간이 -1일 경우 -1로 
        temp = temp[['고객번호', '규모', '종목보유기간']]
        grouped = temp.groupby(['고객번호', '규모'])
        temp = grouped.agg({'종목보유기간': for_grouping})

        temp = temp.pivot_table(index='고객번호', columns='규모', values='종목보유기간').reset_index()
        cust_cap_term = pd.concat([cust_cap_term, temp], ignore_index=True)

    cust_cap_term.fillna(0, inplace=True)
    return cust_cap_term
    # -1은 평균값으로 넣기 --> 매수는 했지만 매도는 없는 형태



In [26]:
cal_term_df = cal_term(file)
cal_term_df = cal_term_df.replace(-1, np.nan)
cal_term_df.fillna(cal_term_df.mean(), inplace=True)
cal_term_df

,고객번호,etf보유기간,우량주보유기간,중형주보유기간,소형주보유기간,kosdaq보유기간
0,C00010,0.000000,10063.583333,0.0,43615.746296,3965.972222
1,C0001,30784.645370,0.000000,0.0,0.000000,0.000000
2,C000100,8460.383333,83.563333,0.0,0.000000,0.000000


In [27]:
customer_info = pd.merge(customer_info, cal_term_df, on='고객번호')
customer_info

,고객번호,고객성별구분코드,동일나이군구분코드,주소,상품개수,총매수금액,총매수수량,ETF,kosdaq_cap,large_cap,medium_cap,small_cap,Industry,etf보유기간,우량주보유기간,중형주보유기간,소형주보유기간,kosdaq보유기간
0,C0001,1,10,경기 성남시,3,8814255,1327,1.000000,0.000000,0.000000,0.000000,0.00000,0.0,30784.645370,0.000000,0.0,0.000000,0.000000
1,C00010,1,30,충남 아산시,9,51683575,23704,0.000000,0.521941,0.327249,0.000000,0.15081,9.0,0.000000,10063.583333,0.0,43615.746296,3965.972222
2,C000100,1,50,서울 강서구,6,48478705,2487,0.202753,0.000000,0.698740,0.098507,0.00000,5.0,8460.383333,83.563333,0.0,0.000000,0.000000


### 6. 수익률: (총매도금액/총매수금액 -1)
매도를 하지 않은 경우의 수익률은 어떻게 계산할지 조금 더 고민→ 4월 1일 매도 기준 

In [28]:
# 수익률 계산 
def cal_rate_of_return(df):
    cust_acc_set = set(df['고객번호'])
    cust_rate_return = pd.DataFrame(columns=['고객번호', '수익률'])
    
    for cust in cust_acc_set:
        invest = df[df['고객번호'] == cust][['고객번호', '계좌번호', '주문일자', '주문시각', '최초주문번호', '주문번호', '상품번호', '상품명', '거래소구분코드', '매도매수구분코드', '실주문단가', '주문수량', '정정취소구분코드', '정정확인수량', '취소확인수량', '총체결건수', '총체결수량', '총체결금액']]
        combine_df = pd.DataFrame(columns= invest.columns)

        # (고객번호)가 거래한 상품번호
        stock_code = invest['상품번호'].unique().tolist()
        
        total_sell_price = 0
        total_buy_price = 0
        
        # loop 한번 돌 때 마다 상품번호 끼리 묶음 
        for num in stock_code:
            temp = invest.loc[invest['상품번호'] == num]

            # 주문취소 row 2개 삭제
            cancel_set = set(temp[temp['정정취소구분코드'] == 2]['최초주문번호'])
            temp = temp[~temp['최초주문번호'].isin(cancel_set)]

            # 가격 정정 --> 한줄로 (마지막으로 체결된 row만 남김)
            correct_set = set(temp[temp['정정취소구분코드'] == 1]['최초주문번호'])
            temp = temp.drop(temp[(temp['최초주문번호'].isin(correct_set)) & (temp['총체결수량'] == 0)].index)
            
            
            # 4월 1일 종가 가져오기
            ticker_code = temp['상품번호'].apply(lambda x: x[6:])
            closing_price = stock.get_market_ohlcv_by_date("20200401", "20200401", ticker_code).iloc[0]['종가']
            
            # 매도매수수량
            sell_quantity = temp[temp['매도매수구분코드']==1]['총체결수량'].sum()
            buy_quantity = temp[temp['매도매수구분코드']==2]['총체결수량'].sum()
            
            
            # 총매수매도금액 
            buy_price = temp[temp['매도매수구분코드']==2]['총체결금액'].sum()
            sell_price = temp[temp['매도매수구분코드']==1]['총체결금액'].sum()
            sell_price = sell_price + (closing_price*(buy_quantity-sell_quantity))
            total_sell_price += sell_price
            total_buy_price += buy_price
        
        # 수익률
        rate_of_return = (total_sell_price/total_buy_price -1) * 100

        cust_rate_return = cust_rate_return.append({'고객번호':cust, '수익률':rate_of_return}, ignore_index=True)
    
    return cust_rate_return            

In [29]:
customer_info = pd.merge(customer_info, cal_rate_of_return(file), on='고객번호')
customer_info

,고객번호,고객성별구분코드,동일나이군구분코드,주소,상품개수,총매수금액,총매수수량,ETF,kosdaq_cap,large_cap,medium_cap,small_cap,Industry,etf보유기간,우량주보유기간,중형주보유기간,소형주보유기간,kosdaq보유기간,수익률
0,C0001,1,10,경기 성남시,3,8814255,1327,1.000000,0.000000,0.000000,0.000000,0.00000,0.0,30784.645370,0.000000,0.0,0.000000,0.000000,-6.683991
1,C00010,1,30,충남 아산시,9,51683575,23704,0.000000,0.521941,0.327249,0.000000,0.15081,9.0,0.000000,10063.583333,0.0,43615.746296,3965.972222,4.066505
2,C000100,1,50,서울 강서구,6,48478705,2487,0.202753,0.000000,0.698740,0.098507,0.00000,5.0,8460.383333,83.563333,0.0,0.000000,0.000000,2.736818


## PER, PBR, ROE, PSR, PEG
7. PER: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균 
8. PBR: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균
9. ROE: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균
10. PSR: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균
11. PEG: 대 /중 / 소 / KOSDAQ 각각의 총매수금액기준 가중평균


In [30]:
# PER, PBR, ROE
customer_info = customer_info.set_index('고객번호')
company_idx = file.loc[file['상품명'].str.contains('투자신탁')== False, '상품번호'].index
file.loc[company_idx,'PER'] = [stock.get_market_fundamental_by_date(file['주문일시'][i],file['주문일시'][i], file['code'][i])['PER'] for i in company_idx ]
file.loc[company_idx,'PBR'] = [stock.get_market_fundamental_by_date(file['주문일시'][i],file['주문일시'][i], file['code'][i])['PBR'] for i in company_idx ]
file.loc[company_idx,'ROE'] = file.loc[company_idx,'PER']/file.loc[company_idx,'PBR']

#고객의 상품별 구매금액 비중
product_purchase_rate = product_purchase.apply(lambda x: x/customer_purchase )

for col in ['PER','PBR','ROE']:
  #고객의 상품별 PER(PBR, ROE) 평균
  product_col = file.groupby(['고객번호','code','stock_type'])[col].mean().unstack().fillna(0)
  
  #고객별 종목타입별 PBR, ROE 가중평균
  col_rate = product_col* product_purchase_rate
  type_col = col_rate.groupby('고객번호').sum()

  customer_info = customer_info.join(type_col, rsuffix='_'+col).drop('ETF_'+col, axis=1)

In [ ]:
# ## PEG -- EPS가 (-)일 때 0으로 기록되어 있어서 직접 EPS부터 구해야 함
# stock.get_market_fundamental_by_date(file['주문일시'][38],file['주문일시'][38], file['code'][38])['EPS']
# stock.get_market_fundamental_by_date(file['주문일시'][38]-datetime.timedelta(weeks = 156),file['주문일시'][38]-datetime.timedelta(weeks = 156), file['code'][38])['EPS']
# # dart.finstate(file['상품번호'][38], '2020','11011')

### 12. 배당율: 대 /중 / 소 / KOSDAQ 각각의 거래액기준 가중평균


In [31]:
# ETF이외의 주식상품 필터링
company_stocks = file.loc[file['상품명'].str.contains('투자신탁')== False, '상품번호'].str[-6:].unique()

baedang_r = pd.DataFrame({'code': company_stocks, 'baedang_r' : 0})

# 현금배당률 뽑아오기
for code in company_stocks:
  baedang = dart.report(code, '배당', 2020)
  baedang_r.loc[baedang_r['code']==code,'baedang_r'] = baedang.iloc[7]['thstrm']
baedang_r = baedang_r.replace('-',0)

# 뽑아온 배당률 데이터프레임을 원래 데이터에 붙이기
file2 = pd.merge(file, baedang_r, how='outer',on='code')
file2['baedang_r'] = file2['baedang_r'].astype('float')

#고객별 종목타입별 배당률 가중평균
product_col = file2.groupby(['고객번호','상품번호','stock_type'])['baedang_r'].mean().unstack().fillna(0)

col_rate = product_col* product_purchase_rate
type_col = col_rate.groupby('고객번호').sum()

customer_info = customer_info.join(type_col, rsuffix='_'+'DY').drop('ETF_'+'DY', axis=1)

### 13. 외국인 매매, 기관매매와의 매도/매수 일치 횟수 (%)

In [ ]:
# pykrx 모듈 에러

# stock.get_exhaustion_rates_of_foreign_investment_by_date(file['주문일시'][38],file['주문일시'][38], file['상품번호'][38])
# help(stock.get_exhaustion_rates_of_foreign_investment_by_date)
# stock.get_exhaustion_rates_of_foreign_investment_by_date("20200120", "20200120", "005930")

##### ETF 제외 -외국인,기관 순매수량

In [32]:

file.loc[company_idx,'외국인_매도매수구분코드'] = [stock.get_market_trading_value_by_date(file['주문일시'][i],file['주문일시'][i], file['code'][i])['외국인합계'] for i in company_idx ]
file.loc[company_idx,'기관_매도매수구분코드'] = [stock.get_market_trading_value_by_date(file['주문일시'][i],file['주문일시'][i], file['code'][i])['기관합계'] for i in company_idx ]

##### ETF 외국인, 기관 순매수량 끌어오기- 크롤링


In [ ]:
# import pandas as pd
# import requests
# import time
# import random

# etf_tickers= stock.get_etf_ticker_list('20200120')  #20200331과 똑같음
# headers = {"user-agent":"Mozilla/5.0"}
# df = pd.DataFrame()

# for i in range(len(etf_tickers)):
#     for j in range(16, 19):
#         url = "https://finance.naver.com/item/frgn.nhn?code=" + etf_tickers[i] + "&page="+str(j)
    
    
#         resp = requests.get(url, headers = headers)
#         resp.text

#         pd_read = pd.read_html(resp.text)
        
#         #for i in range(len(pd_read)):
        
#         new_df = pd_read[2].dropna(axis=0)
#         # new_df = new_df.dropna(axis=0)
#         new_df.insert(0, "종목코드", etf_tickers[i])
#         df = pd.concat([df, new_df])
        
#     time.sleep(random.uniform(2,4))
    
    
# display(df)



##### ETF - 외국인, 기관 순매수량

In [33]:
ETF_investors = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/투자자별순매매_ETF.csv')

ETF_investors.iloc[0,0]= '종목코드'
ETF_investors.iloc[0,6] = '기관_순매매량'
ETF_investors.iloc[0,7] = '외국인_순매매량'

ETF_investors.columns = ETF_investors.iloc[0,:]
ETF_investors.drop(0, axis=0, inplace=True)

##type변환
ETF_investors['날짜'] = pd.to_datetime(ETF_investors['날짜'] ) 
ETF_investors['기관_순매매량']= ETF_investors['기관_순매매량'].astype(float)
ETF_investors['외국인_순매매량'] = ETF_investors['외국인_순매매량'].astype(float)

In [34]:
ETF_investors['종목코드'] = ETF_investors['종목코드'].astype(int).apply(lambda x: '{0:06d}'.format(x)).astype(str)

In [35]:
ETF_idx = file.loc[file['stock_type']== 'ETF'].index
file.loc[ETF_idx,'외국인_매도매수구분코드'] = [ETF_investors.loc[(ETF_investors['종목코드']== file['code'][i])& (ETF_investors['날짜']==file['주문일시'][i].strftime('%Y-%m-%d'))]['외국인_순매매량']for i in ETF_idx]
file.loc[ETF_idx,'기관_매도매수구분코드'] = [ETF_investors.loc[(ETF_investors['종목코드']== file['code'][i])& (ETF_investors['날짜']==file['주문일시'][i].strftime('%Y-%m-%d'))]['기관_순매매량'] for i in ETF_idx]

#### 매도매수구분코드, 매도매수일치비율

In [36]:
file['외국인_매도매수구분코드'] = file['외국인_매도매수구분코드'].apply(lambda x: 1 if x<0 else (2 if x>0 else 0))
file['기관_매도매수구분코드'] = file['기관_매도매수구분코드'].apply(lambda x: 1 if x<0 else (2 if x>0 else 0))
customer_info['매도매수일치_외국인'] = file.loc[file['외국인_매도매수구분코드'] ==file['매도매수구분코드'],['고객번호','매도매수구분코드']].groupby('고객번호').count().apply(lambda x: x/file.groupby('고객번호')['매도매수구분코드'].count())
customer_info['매도매수일치_기관'] = file.loc[file['기관_매도매수구분코드'] ==file['매도매수구분코드'],['고객번호','매도매수구분코드']].groupby('고객번호').count().apply(lambda x: x/file.groupby('고객번호')['매도매수구분코드'].count())

In [37]:
customer_info.head()

,고객성별구분코드,동일나이군구분코드,주소,상품개수,총매수금액,총매수수량,ETF,kosdaq_cap,large_cap,medium_cap,small_cap,Industry,etf보유기간,우량주보유기간,중형주보유기간,소형주보유기간,kosdaq보유기간,수익률,kosdaq_cap_PER,large_cap_PER,medium_cap_PER,small_cap_PER,kosdaq_cap_PBR,large_cap_PBR,medium_cap_PBR,small_cap_PBR,kosdaq_cap_ROE,large_cap_ROE,medium_cap_ROE,small_cap_ROE,kosdaq_cap_DY,large_cap_DY,medium_cap_DY,small_cap_DY,매도매수일치_외국인,매도매수일치_기관
고객번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C0001,1,10,경기 성남시,3,8814255,1327,1.000000,0.000000,0.000000,0.000000,0.00000,0.0,30784.645370,0.000000,0.0,0.000000,0.000000,-6.683991,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.078947,0.289474
C00010,1,30,충남 아산시,9,51683575,23704,0.000000,0.521941,0.327249,0.000000,0.15081,9.0,0.000000,10063.583333,0.0,43615.746296,3965.972222,4.066505,167.078559,1.575159,0.000000,8.169084,13.490437,0.039270,0.000000,0.635664,7.242244,13.091378,0.000000,8.565166,0.331454,0.425424,0.000000,0.0,0.516129,0.483871
C000100,1,50,서울 강서구,6,48478705,2487,0.202753,0.000000,0.698740,0.098507,0.00000,5.0,8460.383333,83.563333,0.0,0.000000,0.000000,2.736818,0.000000,27.167003,0.619611,0.000000,0.000000,2.655211,0.161552,0.000000,0.000000,47.310971,0.377811,0.000000,0.000000,2.289519,0.088657,0.0,0.466667,0.533333


### ATR

In [38]:
file['TR'] = [max(file['전일종가'][i],file['일중고가'][i])- min(file['전일종가'][i],file['일중저가'][i]) for i in file.index]

In [39]:
ohlcv = pd.read_csv('/content/drive/MyDrive/소회의실 프로젝트/ohlcv_2020.csv', encoding = 'euc-kr')
ohlcv['종목코드'] = ohlcv['종목코드'].astype(int).apply(lambda x: '{0:06d}'.format(x)).astype(str)

In [40]:
ETF_code= ['069500', '228790', '305720']
for code in ETF_code:
  ETF_ohlcv =stock.get_market_ohlcv_by_date('20200101', '20200331', code)
  ETF_ohlcv['전일종가'] = ETF_ohlcv['종가'].shift(1)
  ETF_ohlcv = ETF_ohlcv.reset_index()
  ETF_ohlcv.날짜=ETF_ohlcv.날짜.astype(str)
  ETF_ohlcv.insert(0, "종목코드",code)
  ohlcv = pd.concat([ohlcv,ETF_ohlcv]) 

In [41]:
ohlcv =ohlcv.reset_index(drop =True)

In [42]:
ohlcv['TR'] = [max(ohlcv['전일종가'][i],ohlcv['고가'][i])- min(ohlcv['전일종가'][i],ohlcv['저가'][i]) for i in ohlcv.index]
ohlcv = ohlcv.set_index('날짜')
ATR_7 = pd.DataFrame(ohlcv.groupby('종목코드')['TR'].rolling(7).mean())
ATR_14 = pd.DataFrame(ohlcv.groupby('종목코드')['TR'].rolling(14).mean())

ATR_7.columns =['ATR_7']
ATR_14.columns =['ATR_14']

ohlcv = pd.merge(ohlcv, pd.concat([ATR_7,ATR_14], axis=1), on= ['종목코드','날짜'])

In [43]:
file['ATR_7'] = [ohlcv.loc[(ohlcv['종목코드']== file['code'][i])& (ohlcv.index==file['주문일시'][i].strftime('%Y-%m-%d'))]['ATR_7'].values[0] for i in range(len(file))]
file['ATR_14'] = [ohlcv.loc[(ohlcv['종목코드']== file['code'][i])& (ohlcv.index==file['주문일시'][i].strftime('%Y-%m-%d'))]['ATR_14'].values[0] for i in range(len(file))]


In [44]:
for col in ['TR','ATR_7','ATR_14']:
  #고객의 상품별 TR(ATR) 평균
  product_col = file.groupby(['고객번호','code','stock_type'])[col].mean().unstack().fillna(0)
  
  #고객별 종목타입별 TR(ATR) 가중평균
  col_rate = product_col* product_purchase_rate
  type_col = col_rate.groupby('고객번호').sum()

  customer_info = customer_info.join(type_col, rsuffix='_'+col)


In [47]:
customer_info.to_csv('customer_info.csv', encoding='cp949')

In [ ]:
# customer_info.to_csv('고객정보.csv', encoding= 'cp949')


14. 계좌개수: 계좌개수 
15. 시간대: ? -> 나중에 plot 시각화로 판단
16. 주문증거금율: 평균치-- 대출
17. 반대매매횟수
18. 예약주문 횟수
19. 상한가,하한가 여부 횟수: 
주식을 산 날의 변동성을 보기위한 지표 

 
20. 정정/취소 횟수
21. 공모주 비중: 거래액 기준
21. 인버스,곱버스, 레버리지 비중: 거래액 기준 
